# Position Sizing User Guide

This notebook provides a comprehensive guide to using the `PositionSizing` module from the `algoshort` package.

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [Understanding Position Sizing Concepts](#2-understanding-position-sizing-concepts)
3. [PositionSizing Class Configuration](#3-positionsizing-class-configuration)
4. [Basic Usage - Single Signal](#4-basic-usage---single-signal)
5. [Risk Appetite Calculation](#5-risk-appetite-calculation)
6. [Multiple Strategies Comparison](#6-multiple-strategies-comparison)
7. [Parallel Processing for Multiple Signals](#7-parallel-processing-for-multiple-signals)
8. [Parallel Processing Over Multiple Stocks](#8-parallel-processing-over-multiple-stocks)
9. [Complete Workflow Integration](#9-complete-workflow-integration)
10. [Best Practices and Tips](#10-best-practices-and-tips)

## 1. Setup and Installation

First, let's import the required modules and set up logging for better visibility.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import logging

# Configure logging to see debug messages
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import algoshort modules
from algoshort.position_sizing import (
    PositionSizing,
    get_signal_column_names,
    run_position_sizing_parallel,
    run_position_sizing_parallel_over_stocks,
)

print("Imports successful!")

## 2. Understanding Position Sizing Concepts

### What is Position Sizing?

Position sizing determines **how many shares** to buy or sell based on:
- Your account equity
- Risk tolerance (how much you're willing to lose per trade)
- Stop-loss distance from entry price

### The Four Strategies

The `PositionSizing` class implements four different position sizing strategies:

| Strategy | Description | Risk Adjustment |
|----------|-------------|----------------|
| **Equal Weight** | Fixed percentage of equity | None - constant allocation |
| **Constant Risk** | Fixed risk percentage | None - constant risk |
| **Concave** | Risk decreases faster during drawdowns | Conservative - reduces risk quickly |
| **Convex** | Risk decreases slower during drawdowns | Aggressive - maintains higher risk |

### Key Formula: Equity Risk Shares

```
shares = (equity * risk_percentage) / abs(entry_price - stop_loss)
```

This ensures you never lose more than `risk_percentage` of your equity on any single trade.

## 3. PositionSizing Class Configuration

### Constructor Parameters

| Parameter | Type | Description | Constraints |
|-----------|------|-------------|-------------|
| `tolerance` | float | Maximum drawdown tolerance | Must be negative (e.g., -0.15 for 15%) |
| `mn` | float | Minimum risk percentage | Must be positive, <= mx |
| `mx` | float | Maximum risk percentage | Must be positive, >= mn |
| `equal_weight` | float | Equal weight allocation factor | Must be in (0, 1] |
| `avg` | float | Average/constant risk value | Used for constant strategy |
| `lot` | int | Minimum lot size | Must be positive |
| `initial_capital` | float | Starting capital per strategy | Must be positive, default=100000 |

In [ ]:
# Example: Create a PositionSizing instance
sizer = PositionSizing(
    tolerance=-0.15,      # Max 15% drawdown tolerance
    mn=0.01,              # Minimum risk: 1%
    mx=0.03,              # Maximum risk: 3%
    equal_weight=0.10,    # 10% equal weight allocation
    avg=0.02,             # 2% constant risk
    lot=1,                # Minimum 1 share
    initial_capital=100000  # $100,000 starting capital
)

print(f"PositionSizing configured:")
print(f"  Tolerance: {sizer.tolerance} ({abs(sizer.tolerance)*100:.0f}% max drawdown)")
print(f"  Risk range: {sizer.mn*100:.1f}% to {sizer.mx*100:.1f}%")
print(f"  Constant risk: {sizer.avg*100:.1f}%")
print(f"  Equal weight: {sizer.equal_weight*100:.1f}%")
print(f"  Lot size: {sizer.lot}")
print(f"  Initial capital: ${sizer.initial_capital:,.0f}")

In [ ]:
# Parameter validation examples
print("Parameter validation demonstrations:\n")

# Example 1: Invalid tolerance (must be negative)
try:
    bad_sizer = PositionSizing(
        tolerance=0.15,  # ERROR: should be negative
        mn=0.01, mx=0.03, equal_weight=0.10, avg=0.02, lot=1
    )
except ValueError as e:
    print(f"1. Positive tolerance: {e}")

# Example 2: Invalid risk range (mn > mx)
try:
    bad_sizer = PositionSizing(
        tolerance=-0.15,
        mn=0.05, mx=0.02,  # ERROR: mn > mx
        equal_weight=0.10, avg=0.02, lot=1
    )
except ValueError as e:
    print(f"2. Invalid risk range: {e}")

# Example 3: Invalid equal_weight
try:
    bad_sizer = PositionSizing(
        tolerance=-0.15, mn=0.01, mx=0.03,
        equal_weight=1.5,  # ERROR: > 1
        avg=0.02, lot=1
    )
except ValueError as e:
    print(f"3. Invalid equal_weight: {e}")

# Example 4: Invalid lot size
try:
    bad_sizer = PositionSizing(
        tolerance=-0.15, mn=0.01, mx=0.03, equal_weight=0.10,
        avg=0.02, lot=0  # ERROR: must be positive
    )
except ValueError as e:
    print(f"4. Invalid lot size: {e}")

## 4. Basic Usage - Single Signal

### Creating Sample Data

Let's create a sample DataFrame with the required columns for position sizing.

In [ ]:
# Create sample trading data
np.random.seed(42)
n_days = 252  # One trading year

# Generate price data with trend and noise
dates = pd.date_range('2024-01-01', periods=n_days, freq='B')
returns = np.random.normal(0.0005, 0.02, n_days)  # Daily returns
prices = 100 * np.exp(np.cumsum(returns))  # Geometric random walk

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'close': prices,
    'high': prices * (1 + np.abs(np.random.normal(0, 0.01, n_days))),
    'low': prices * (1 - np.abs(np.random.normal(0, 0.01, n_days))),
})

# Calculate daily change (price difference)
df['chg1D_fx'] = df['close'].diff().fillna(0)

# Generate a simple moving average crossover signal
df['sma_20'] = df['close'].rolling(20).mean()
df['sma_50'] = df['close'].rolling(50).mean()

# Signal: 1 = long, -1 = short, 0 = flat
df['ma_cross'] = 0
df.loc[df['sma_20'] > df['sma_50'], 'ma_cross'] = 1
df.loc[df['sma_20'] < df['sma_50'], 'ma_cross'] = -1

# Calculate ATR for stop-loss
df['tr'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(
        np.abs(df['high'] - df['close'].shift(1)),
        np.abs(df['low'] - df['close'].shift(1))
    )
).fillna(0)
df['atr_14'] = df['tr'].rolling(14).mean().fillna(df['tr'])

# Stop-loss: 2x ATR from close
df['stop_loss'] = df['close'] - (2 * df['atr_14'] * df['ma_cross'].replace(0, 1))

# Drop NaN rows from moving average warmup
df = df.dropna().reset_index(drop=True)

print(f"Sample data shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head(10)

### Using eqty_risk_shares() - Calculate Shares for a Single Trade

In [ ]:
# Calculate shares for a single trade
entry_price = 100.0
stop_loss = 95.0  # $5 risk per share
equity = 100000.0
risk = 0.02  # 2% risk

shares = sizer.eqty_risk_shares(
    px=entry_price,
    sl=stop_loss,
    eqty=equity,
    risk=risk,
    fx=1.0,  # No currency conversion
    lot=1
)

print("Single Trade Position Sizing:")
print(f"  Entry Price: ${entry_price:.2f}")
print(f"  Stop Loss: ${stop_loss:.2f}")
print(f"  Risk per Share: ${abs(entry_price - stop_loss):.2f}")
print(f"  Account Equity: ${equity:,.2f}")
print(f"  Risk Percentage: {risk*100:.1f}%")
print(f"  Max Loss Amount: ${equity * risk:,.2f}")
print(f"  ========================================")
print(f"  Calculated Shares: {shares}")
print(f"  Position Value: ${shares * entry_price:,.2f}")
print(f"  Actual Risk: ${shares * abs(entry_price - stop_loss):,.2f}")

In [ ]:
# Edge cases for eqty_risk_shares
print("Edge Case Demonstrations:\n")

# Case 1: Stop loss very close to entry (high leverage attempt)
shares_tight = sizer.eqty_risk_shares(
    px=100.0, sl=99.999,  # Only $0.001 risk
    eqty=100000, risk=0.02, fx=1.0, lot=1
)
print(f"1. Tight stop loss (SL=99.999): {shares_tight} shares")

# Case 2: Stop loss equals entry price
shares_equal = sizer.eqty_risk_shares(
    px=100.0, sl=100.0,  # Zero risk
    eqty=100000, risk=0.02, fx=1.0, lot=1
)
print(f"2. SL equals price (SL=100.0): {shares_equal} shares")

# Case 3: Short position (stop above entry)
shares_short = sizer.eqty_risk_shares(
    px=100.0, sl=105.0,  # Short: stop above entry
    eqty=100000, risk=0.02, fx=1.0, lot=1
)
print(f"3. Short position (SL=105.0): {shares_short} shares")

# Case 4: With lot size of 100
shares_lot100 = sizer.eqty_risk_shares(
    px=100.0, sl=95.0,
    eqty=100000, risk=0.02, fx=1.0, lot=100
)
print(f"4. Lot size 100: {shares_lot100} shares")

# Case 5: Currency conversion (fx=1.2)
shares_fx = sizer.eqty_risk_shares(
    px=100.0, sl=95.0,
    eqty=100000, risk=0.02, fx=1.2, lot=1
)
print(f"5. With FX=1.2: {shares_fx} shares")

### Using calculate_shares() - Full Portfolio Simulation

In [ ]:
# Run full position sizing calculation
df_result = sizer.calculate_shares(
    df=df.copy(),
    signal='ma_cross',      # Signal column
    daily_chg='chg1D_fx',   # Daily change column
    sl='stop_loss',         # Stop loss column
    close='close'           # Close price column
)

print("New columns added:")
new_cols = [c for c in df_result.columns if c not in df.columns]
for col in new_cols:
    print(f"  - {col}")

In [ ]:
# View equity curves
equity_cols = [c for c in df_result.columns if 'equity' in c]
df_result[['date'] + equity_cols].tail(10)

In [ ]:
# Plot equity curves
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))

for col in equity_cols:
    strategy_name = col.replace('ma_cross_equity_', '').title()
    ax.plot(df_result['date'], df_result[col], label=strategy_name, linewidth=1.5)

ax.set_xlabel('Date')
ax.set_ylabel('Equity ($)')
ax.set_title('Position Sizing Strategies Comparison')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print final equity values
print("\nFinal Equity Values:")
for col in equity_cols:
    strategy_name = col.replace('ma_cross_equity_', '').title()
    final_equity = df_result[col].iloc[-1]
    returns_pct = (final_equity - 100000) / 100000 * 100
    print(f"  {strategy_name:10s}: ${final_equity:>12,.2f} ({returns_pct:+.2f}%)")

## 5. Risk Appetite Calculation

The `risk_appetite()` method calculates dynamic risk based on the equity curve's drawdown.

In [ ]:
# Create a sample equity curve with drawdown
equity_curve = pd.Series([
    100000, 102000, 105000, 103000, 100000,  # Initial growth then drawdown
    98000, 95000, 93000, 96000, 99000,       # Deeper drawdown then recovery
    102000, 105000, 108000, 106000, 110000   # Recovery and new highs
])

print("Sample equity curve:")
print(equity_curve.values)

In [ ]:
# Calculate risk appetite for different shapes
shapes = {
    'Concave (Conservative)': -1,
    'Linear': 0,
    'Convex (Aggressive)': 1
}

risk_results = {}
for name, shape in shapes.items():
    risk_results[name] = sizer.risk_appetite(
        eqty=equity_curve,
        tolerance=-0.15,  # 15% drawdown tolerance
        mn=0.01,          # 1% minimum risk
        mx=0.03,          # 3% maximum risk
        span=5,           # EMA span
        shape=shape
    )

# Create comparison DataFrame
risk_df = pd.DataFrame({
    'Equity': equity_curve,
    **risk_results
})

# Calculate drawdown for reference
risk_df['Drawdown'] = equity_curve / equity_curve.expanding().max() - 1

risk_df

In [ ]:
# Plot risk appetite vs drawdown
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Plot 1: Equity and Drawdown
ax1 = axes[0]
ax1.plot(risk_df['Equity'], 'b-', linewidth=2, label='Equity')
ax1.set_ylabel('Equity ($)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax1_twin = ax1.twinx()
ax1_twin.fill_between(risk_df.index, risk_df['Drawdown'] * 100, 0, 
                       alpha=0.3, color='red', label='Drawdown')
ax1_twin.set_ylabel('Drawdown (%)', color='red')
ax1_twin.tick_params(axis='y', labelcolor='red')
ax1.set_title('Equity Curve with Drawdown')
ax1.legend(loc='upper left')

# Plot 2: Risk Appetite by Shape
ax2 = axes[1]
colors = ['green', 'blue', 'orange']
for (name, _), color in zip(shapes.items(), colors):
    ax2.plot(risk_df[name] * 100, label=name, linewidth=2, color=color)

ax2.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Min Risk (1%)')
ax2.axhline(y=3, color='gray', linestyle='--', alpha=0.5, label='Max Risk (3%)')
ax2.set_xlabel('Period')
ax2.set_ylabel('Risk Appetite (%)')
ax2.set_title('Dynamic Risk Appetite by Shape')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Understanding the Shapes

- **Concave (shape=-1)**: Risk decreases **faster** as drawdown increases. Conservative approach.
- **Linear (shape=0)**: Risk decreases **proportionally** to drawdown. Balanced approach.
- **Convex (shape=1)**: Risk decreases **slower** as drawdown increases. Aggressive approach.

## 6. Multiple Strategies Comparison

Let's compare how different risk configurations affect performance.

In [ ]:
# Create multiple sizer configurations
configs = {
    'Conservative': PositionSizing(
        tolerance=-0.10, mn=0.005, mx=0.015,
        equal_weight=0.05, avg=0.01, lot=1
    ),
    'Moderate': PositionSizing(
        tolerance=-0.15, mn=0.01, mx=0.03,
        equal_weight=0.10, avg=0.02, lot=1
    ),
    'Aggressive': PositionSizing(
        tolerance=-0.25, mn=0.02, mx=0.05,
        equal_weight=0.15, avg=0.035, lot=1
    )
}

print("Configuration Comparison:")
print(f"{'Config':<15} {'Tolerance':<12} {'Risk Range':<15} {'Avg Risk':<10}")
print("-" * 55)
for name, s in configs.items():
    print(f"{name:<15} {s.tolerance*100:>6.0f}%      {s.mn*100:.1f}%-{s.mx*100:.1f}%       {s.avg*100:.1f}%")

In [ ]:
# Run each configuration and compare
results = {}

for name, sizer_config in configs.items():
    df_temp = sizer_config.calculate_shares(
        df=df.copy(),
        signal='ma_cross',
        daily_chg='chg1D_fx',
        sl='stop_loss',
        close='close'
    )
    
    # Store convex equity (most aggressive within each config)
    results[name] = df_temp['ma_cross_equity_convex'].copy()

# Create comparison DataFrame
comparison_df = pd.DataFrame(results)
comparison_df['date'] = df_result['date'].values

comparison_df.tail()

In [ ]:
# Plot comparison
fig, ax = plt.subplots(figsize=(12, 6))

colors = {'Conservative': 'green', 'Moderate': 'blue', 'Aggressive': 'red'}
for name in configs.keys():
    ax.plot(comparison_df['date'], comparison_df[name], 
            label=name, linewidth=2, color=colors[name])

ax.axhline(y=100000, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Equity ($)')
ax.set_title('Risk Configuration Comparison (Convex Strategy)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print statistics
print("\nPerformance Statistics:")
print(f"{'Config':<15} {'Final Equity':<15} {'Return':<10} {'Max DD':<10}")
print("-" * 50)
for name in configs.keys():
    final = comparison_df[name].iloc[-1]
    ret = (final - 100000) / 100000 * 100
    max_dd = (comparison_df[name] / comparison_df[name].expanding().max() - 1).min() * 100
    print(f"{name:<15} ${final:>12,.0f} {ret:>+8.1f}%  {max_dd:>8.1f}%")

## 7. Parallel Processing for Multiple Signals

Use `run_position_sizing_parallel()` to process multiple signals efficiently.

In [ ]:
# Create multiple signals for demonstration
df_multi = df.copy()

# Signal 1: MA Cross (already exists)
# Signal 2: RSI-based signal
delta = df_multi['close'].diff()
gain = delta.where(delta > 0, 0).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
rs = gain / loss.replace(0, np.nan)
df_multi['rsi'] = 100 - (100 / (1 + rs))
df_multi['rsi_signal'] = 0
df_multi.loc[df_multi['rsi'] < 30, 'rsi_signal'] = 1   # Oversold = buy
df_multi.loc[df_multi['rsi'] > 70, 'rsi_signal'] = -1  # Overbought = sell

# Signal 3: Breakout signal
df_multi['upper_20'] = df_multi['high'].rolling(20).max()
df_multi['lower_20'] = df_multi['low'].rolling(20).min()
df_multi['breakout_signal'] = 0
df_multi.loc[df_multi['close'] > df_multi['upper_20'].shift(1), 'breakout_signal'] = 1
df_multi.loc[df_multi['close'] < df_multi['lower_20'].shift(1), 'breakout_signal'] = -1

# Create required columns for each signal
signals = ['ma_cross', 'rsi_signal', 'breakout_signal']

for signal in signals:
    # Daily change column (using close diff as simplified example)
    df_multi[f'{signal}_chg1D_fx'] = df_multi['chg1D_fx']
    
    # Stop loss column (2x ATR)
    df_multi[f'{signal}_stop_loss'] = df_multi['close'] - (
        2 * df_multi['atr_14'] * df_multi[signal].replace(0, 1)
    )

df_multi = df_multi.dropna().reset_index(drop=True)

print(f"DataFrame shape: {df_multi.shape}")
print(f"\nSignals to process: {signals}")
print(f"\nSample of required columns:")
print(df_multi[['close', 'ma_cross', 'ma_cross_chg1D_fx', 'ma_cross_stop_loss']].head())

In [ ]:
# Get column name mapping
for signal in signals:
    cols = get_signal_column_names(
        signal=signal,
        chg_suffix='_chg1D_fx',
        sl_suffix='_stop_loss',
        close_col='close'
    )
    print(f"\n{signal}:")
    for key, value in cols.items():
        print(f"  {key}: {value}")

In [ ]:
# Run parallel position sizing
sizer_parallel = PositionSizing(
    tolerance=-0.15,
    mn=0.01,
    mx=0.03,
    equal_weight=0.10,
    avg=0.02,
    lot=1,
    initial_capital=100000
)

df_parallel = run_position_sizing_parallel(
    sizer=sizer_parallel,
    df=df_multi,
    signals=signals,
    chg_suffix='_chg1D_fx',
    sl_suffix='_stop_loss',
    close_col='close',
    n_jobs=-1,   # Use all CPU cores
    verbose=0    # Reduce output verbosity
)

print(f"\nResult shape: {df_parallel.shape}")
print(f"\nNew columns added ({len(df_parallel.columns) - len(df_multi.columns)}):")

In [ ]:
# Compare final equity across all signals and strategies
print("Final Equity Comparison:")
print("=" * 70)

for signal in signals:
    print(f"\n{signal}:")
    equity_cols = [c for c in df_parallel.columns if signal in c and 'equity' in c]
    for col in equity_cols:
        strategy = col.split('_')[-1]
        final_val = df_parallel[col].iloc[-1]
        ret = (final_val - 100000) / 100000 * 100
        print(f"  {strategy:>10}: ${final_val:>12,.2f} ({ret:>+6.2f}%)")

In [ ]:
# Plot all equity curves
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, signal in enumerate(signals):
    ax = axes[idx]
    equity_cols = [c for c in df_parallel.columns if signal in c and 'equity' in c]
    
    for col in equity_cols:
        strategy = col.split('_')[-1].title()
        ax.plot(df_parallel[col], label=strategy, linewidth=1.5)
    
    ax.axhline(y=100000, color='gray', linestyle='--', alpha=0.5)
    ax.set_title(signal.replace('_', ' ').title())
    ax.set_xlabel('Period')
    ax.set_ylabel('Equity ($)')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Parallel Processing Over Multiple Stocks

When you have a **portfolio of stocks**, processing them sequentially can be slow.  
`run_position_sizing_parallel_over_stocks()` parallelises over the **stock** dimension  
while processing signals **serially** inside each worker — avoiding nested parallelism.

### When to Use Each Approach

| Scenario | Recommended Function |
|----------|---------------------|
| One stock, many signals | `run_position_sizing_parallel()` |
| Many stocks, few or many signals | `run_position_sizing_parallel_over_stocks()` |

**Memory advantage**: each worker receives only *one* stock's DataFrame  
(`O(one_stock_df)`), keeping peak per-process memory low regardless of portfolio size.

**Return type**: `dict[ticker → enriched DataFrame]` — the original DataFrames are  
not modified; each result is an independent copy with position-sizing columns appended.

In [ ]:
# ── Build a portfolio of stock DataFrames ───────────────────────────────────
def make_stock_df(n: int = 252, seed: int = 42, base_price: float = 100.0) -> pd.DataFrame:
    """Generate a synthetic OHLC DataFrame with two MA-crossover signals."""
    rng = np.random.default_rng(seed)
    returns = rng.normal(0.0005, 0.02, n)
    prices  = base_price * np.exp(np.cumsum(returns))
    highs   = prices * (1 + np.abs(rng.normal(0, 0.01, n)))
    lows    = prices * (1 - np.abs(rng.normal(0, 0.01, n)))

    df = pd.DataFrame({"close": prices, "high": highs, "low": lows})
    df["chg1D_fx"] = df["close"].diff().fillna(0)

    # ATR (14-period) for stop-loss distance
    tr  = np.maximum(
        df["high"] - df["low"],
        np.maximum(
            np.abs(df["high"] - df["close"].shift(1)),
            np.abs(df["low"]  - df["close"].shift(1)),
        ),
    ).fillna(0)
    atr = tr.rolling(14).mean().fillna(tr)

    # Two MA-crossover signals with per-signal chg and stop-loss columns
    for sig_name, fast, slow in [("ma_fast", 10, 30), ("ma_slow", 20, 60)]:
        sma_f = df["close"].rolling(fast).mean()
        sma_s = df["close"].rolling(slow).mean()
        sig   = np.sign(sma_f - sma_s).fillna(0).astype(int)
        df[sig_name]                    = sig
        df[f"{sig_name}_chg1D_fx"]      = df["chg1D_fx"]
        df[f"{sig_name}_stop_loss"]     = df["close"] - 2 * atr * sig.replace(0, 1)

    return df.dropna().reset_index(drop=True)


tickers     = ["ENI.MI", "ENEL.MI", "ISP.MI"]
seeds       = [42, 7, 13]
base_prices = [14.5, 6.8, 3.2]

stock_dfs = {
    ticker: make_stock_df(seed=seed, base_price=bp)
    for ticker, seed, bp in zip(tickers, seeds, base_prices)
}

print(f"Portfolio: {list(stock_dfs.keys())}")
for ticker, df_s in stock_dfs.items():
    print(f"  {ticker}: {df_s.shape[0]} rows, {df_s.shape[1]} columns")

In [ ]:
# ── Run stock-parallel position sizing ─────────────────────────────────────
portfolio_sizer = PositionSizing(
    tolerance=-0.15,
    mn=0.01,
    mx=0.03,
    equal_weight=0.10,
    avg=0.02,
    lot=1,
    initial_capital=100_000,
)

signals = ["ma_fast", "ma_slow"]

stock_results = run_position_sizing_parallel_over_stocks(
    sizer=portfolio_sizer,
    stock_dfs=stock_dfs,
    signals=signals,
    chg_suffix="_chg1D_fx",
    sl_suffix="_stop_loss",
    close_col="close",
    n_jobs=1,   # n_jobs=1 is safest inside notebooks on all platforms;
                # set n_jobs=-1 to use all cores in a standalone script
)

print(f"Processed {len(stock_results)} tickers: {list(stock_results.keys())}")
for ticker, df_r in stock_results.items():
    new_cols = [c for c in df_r.columns if c not in stock_dfs[ticker].columns]
    print(f"  {ticker}: {len(new_cols)} new columns added")

In [ ]:
# ── Inspect final equity per ticker ────────────────────────────────────────
print(f"{'='*68}")
print(f"{'Ticker':<12}  {'Signal':<10}  {'Strategy':<10}  {'Final Equity':>14}  {'Return':>8}")
print(f"{'='*68}")

for ticker, df_r in stock_results.items():
    equity_cols = sorted(c for c in df_r.columns if "equity" in c)
    for col in equity_cols:
        # Column format: <signal>_equity_<strategy>
        parts = col.split("_equity_")
        signal_name = parts[0] if len(parts) == 2 else col
        strategy    = parts[1] if len(parts) == 2 else ""
        final_val   = df_r[col].iloc[-1]
        ret         = (final_val - 100_000) / 100_000 * 100
        print(f"{ticker:<12}  {signal_name:<10}  {strategy:<10}  ${final_val:>13,.2f}  {ret:>+7.2f}%")
    print()

In [ ]:
# ── Cross-stock equity comparison ──────────────────────────────────────────
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, len(tickers), figsize=(15, 5), sharey=False)

for ax, ticker in zip(axes, tickers):
    df_r = stock_results[ticker]
    equity_cols = sorted(c for c in df_r.columns if "equity_convex" in c)
    for col in equity_cols:
        signal_name = col.replace("_equity_convex", "")
        ax.plot(df_r[col], label=signal_name, linewidth=1.5)
    ax.axhline(y=100_000, color="gray", linestyle="--", alpha=0.5)
    ax.set_title(ticker)
    ax.set_xlabel("Period")
    ax.set_ylabel("Equity ($)")
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle("Convex Equity Curves — Stock-Parallel Position Sizing", y=1.02)
plt.tight_layout()
plt.show()

## 9. Complete Workflow Integration

Here's how to integrate position sizing with other `algoshort` modules.

In [ ]:
# Complete workflow example
from algoshort.yfinance_handler import YFinanceDataHandler
from algoshort.ohlcprocessor import OHLCProcessor
from algoshort.stop_loss import StopLossCalculator
from algoshort.returns import ReturnsCalculator

print("All modules imported successfully!")

In [ ]:
# Step 1: Download data
handler = YFinanceDataHandler()
symbol = 'SPY'

df_raw = handler.get_ohlc_data(symbol, period='2y', interval='1d')
print(f"Downloaded {len(df_raw)} rows for {symbol}")
df_raw.head()

In [ ]:
# Step 2: Process OHLC data
processor = OHLCProcessor()
df_processed = processor.process(df_raw.copy())
print(f"Processed columns: {list(df_processed.columns)}")

In [ ]:
# Step 3: Generate a trading signal
df_signal = df_processed.copy()

# Simple momentum signal: price above 50-day SMA = long
df_signal['sma_50'] = df_signal['close'].rolling(50).mean()
df_signal['momentum'] = 0
df_signal.loc[df_signal['close'] > df_signal['sma_50'], 'momentum'] = 1
df_signal.loc[df_signal['close'] < df_signal['sma_50'], 'momentum'] = -1

df_signal = df_signal.dropna().reset_index(drop=True)
print(f"Signal distribution:\n{df_signal['momentum'].value_counts()}")

In [ ]:
# Step 4: Calculate stop-loss
stop_calc = StopLossCalculator(df_signal)
df_stops = stop_calc.atr_stop_loss(
    signal='momentum',
    multiplier=2.0,
    window=14,
    forward_fill=True
)

print(f"Stop-loss column added: momentum_atr_stop_loss")
df_stops[['close', 'momentum', 'momentum_atr_stop_loss']].tail()

In [ ]:
# Step 5: Calculate returns
returns_calc = ReturnsCalculator()
df_returns = returns_calc.calculate_returns(
    df=df_stops.copy(),
    signals=['momentum'],
    inplace=False
)

# Rename for position sizing compatibility
df_returns['momentum_chg1D_fx'] = df_returns['close'].diff().fillna(0)
df_returns['momentum_stop_loss'] = df_returns['momentum_atr_stop_loss']

df_returns = df_returns.dropna().reset_index(drop=True)
print(f"Returns calculated. Shape: {df_returns.shape}")

In [ ]:
# Step 6: Run position sizing
portfolio_sizer = PositionSizing(
    tolerance=-0.15,
    mn=0.01,
    mx=0.03,
    equal_weight=0.10,
    avg=0.02,
    lot=1,
    initial_capital=100000
)

df_final = portfolio_sizer.calculate_shares(
    df=df_returns,
    signal='momentum',
    daily_chg='momentum_chg1D_fx',
    sl='momentum_stop_loss',
    close='close'
)

print("Position sizing complete!")

In [ ]:
# Final results visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Plot 1: Price and Signal
ax1 = axes[0]
ax1.plot(df_final.index, df_final['close'], 'b-', label='Close', linewidth=1)
ax1.plot(df_final.index, df_final['sma_50'], 'orange', label='SMA 50', linewidth=1)

# Shade long/short periods
long_mask = df_final['momentum'] == 1
short_mask = df_final['momentum'] == -1
ax1.fill_between(df_final.index, df_final['close'].min(), df_final['close'].max(),
                 where=long_mask, alpha=0.2, color='green', label='Long')
ax1.fill_between(df_final.index, df_final['close'].min(), df_final['close'].max(),
                 where=short_mask, alpha=0.2, color='red', label='Short')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{symbol} - Price with Momentum Signal')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Equity Curves
ax2 = axes[1]
equity_cols = [c for c in df_final.columns if 'momentum_equity' in c]
for col in equity_cols:
    strategy = col.replace('momentum_equity_', '').title()
    ax2.plot(df_final.index, df_final[col], label=strategy, linewidth=1.5)
ax2.axhline(y=100000, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Equity ($)')
ax2.set_title('Portfolio Equity by Strategy')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

# Plot 3: Shares Position
ax3 = axes[2]
shares_cols = [c for c in df_final.columns if 'momentum_shares' in c]
for col in shares_cols:
    strategy = col.replace('momentum_shares_', '').title()
    ax3.plot(df_final.index, df_final[col], label=strategy, linewidth=1)
ax3.set_xlabel('Period')
ax3.set_ylabel('Shares')
ax3.set_title('Position Size by Strategy')
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Performance summary
print(f"\n{'='*60}")
print(f"PERFORMANCE SUMMARY - {symbol}")
print(f"{'='*60}")
print(f"\nPeriod: {df_final.index[0]} to {df_final.index[-1]} ({len(df_final)} days)")
print(f"Initial Capital: $100,000")
print(f"\n{'Strategy':<15} {'Final Equity':>15} {'Return':>10} {'Max DD':>10}")
print("-" * 55)

for col in equity_cols:
    strategy = col.replace('momentum_equity_', '').title()
    final_eq = df_final[col].iloc[-1]
    total_ret = (final_eq - 100000) / 100000 * 100
    max_dd = (df_final[col] / df_final[col].expanding().max() - 1).min() * 100
    print(f"{strategy:<15} ${final_eq:>13,.2f} {total_ret:>+9.2f}% {max_dd:>9.2f}%")

# Buy and hold comparison
initial_price = df_final['close'].iloc[0]
final_price = df_final['close'].iloc[-1]
bh_return = (final_price - initial_price) / initial_price * 100
print(f"\n{'Buy & Hold':<15} ${100000 * (1 + bh_return/100):>13,.2f} {bh_return:>+9.2f}%")

## 10. Best Practices and Tips

### Parameter Guidelines

| Parameter | Conservative | Moderate | Aggressive |
|-----------|-------------|----------|------------|
| `tolerance` | -0.10 | -0.15 | -0.25 |
| `mn` (min risk) | 0.5% | 1% | 2% |
| `mx` (max risk) | 1.5% | 3% | 5% |
| `equal_weight` | 5% | 10% | 15% |
| `avg` | 1% | 2% | 3.5% |

### Common Pitfalls

1. **Stop-loss too tight**: If `abs(sl - px)` is very small, position size becomes very large
2. **Inconsistent column naming**: Ensure signal columns match the expected naming convention
3. **Missing data**: Always handle NaN values before running position sizing
4. **Lot size mismatch**: Some markets require specific lot sizes (e.g., 100 for stocks)

### Memory Optimization

For large datasets:

In [ ]:
# Use calculate_shares_for_signal for thread-safe operations
df_safe = sizer.calculate_shares_for_signal(
    df=df.copy(),
    signal='ma_cross',
    daily_chg='chg1D_fx',
    sl='stop_loss',
    close='close'
)

# Original DataFrame is not modified
print(f"Original columns: {len(df.columns)}")
print(f"Result columns: {len(df_safe.columns)}")

### Saving Results

In [ ]:
# Save to various formats
import os

# Create output directory
output_dir = 'output/position_sizing'
os.makedirs(output_dir, exist_ok=True)

# Save to Parquet (recommended for large datasets)
df_final.to_parquet(f'{output_dir}/portfolio_results.parquet')

# Save to CSV
df_final.to_csv(f'{output_dir}/portfolio_results.csv', index=False)

# Save only equity columns
equity_df = df_final[['close'] + equity_cols]
equity_df.to_csv(f'{output_dir}/equity_curves.csv', index=False)

print(f"Results saved to {output_dir}/")
print(f"  - portfolio_results.parquet")
print(f"  - portfolio_results.csv")
print(f"  - equity_curves.csv")

### Quick Reference

```python
# Basic usage
from algoshort.position_sizing import PositionSizing, run_position_sizing_parallel

# Create sizer
sizer = PositionSizing(
    tolerance=-0.15,    # 15% max drawdown
    mn=0.01, mx=0.03,   # 1-3% risk range
    equal_weight=0.10,  # 10% equal weight
    avg=0.02,           # 2% constant risk
    lot=1               # Min lot size
)

# Single signal
result = sizer.calculate_shares(df, signal='my_signal', ...)

# Multiple signals in parallel
result = run_position_sizing_parallel(
    sizer, df, signals=['sig1', 'sig2'], n_jobs=-1
)

# Calculate shares for a single trade
shares = sizer.eqty_risk_shares(px=100, sl=95, eqty=100000, risk=0.02, fx=1, lot=1)
```

---

## Summary

This guide covered:

1. **Setup**: Import and configure the PositionSizing module
2. **Concepts**: Understanding the four position sizing strategies
3. **Configuration**: Parameter validation and constraints
4. **Basic Usage**: Single trade and full portfolio calculations
5. **Risk Appetite**: Dynamic risk adjustment based on drawdown
6. **Strategy Comparison**: Conservative vs Moderate vs Aggressive
7. **Parallel Processing (Signals)**: Efficiently handle multiple signals in parallel
8. **Parallel Processing (Stocks)**: Process a portfolio of stocks in parallel with lower per-worker memory usage
9. **Integration**: Complete workflow with other algoshort modules
10. **Best Practices**: Tips for production use

For questions or issues, refer to the test suite at `tests/test_position_sizing.py`.